In [1]:
! pip install faiss-cpu==1.7.4 mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.5/262.5 kB 14.4 MB/s eta 0:00:00


In [2]:
from mistralai import Mistral
import requests
import numpy as np
import faiss
import os
from getpass import getpass

api_key= getpass("Type your API Key")
client = Mistral(api_key=api_key)

Type your API Key··········


### Get data

Загружаю часть лекций Линейной Алгебре из файла lektsii.txt (сами лекции были взяты из https://github.com/HechenHu/Mathematics-Notes/blob/master/Linear%20Algebra/Linear_Algebra.tex)


In [13]:
with open('lektsii.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [14]:
len(text)

65319

In [18]:
print(text[:2048])


\chapter{Vector Spaces}
\section{Definitions}
\begin{Definition}
		A \textit{vector space} over a field $ F $ is a set $ V $ with two closed operations, \textit{vector addition} or simply \textit{addition} and \textit{scalar multiplication}, that satisfy the following axioms:
		\begin{enumerate}
			\item Associativity of addition;
			\item Commutativity of addition;
			\item Identity element of addition;
			\item Inverse elements of addition;
			\item Compatibility of scalar multiplication with field multiplication;
			\begin{equation}
			a(b \vec{v})=(ab)\vec{v} \nonumber
			\end{equation}
			\item Identity element of scalar multiplication;
			\item Distributivity of scalar multiplication with respect to vector addition;
			\item Distributivity of scalar multiplication with respect to field addition.
		\end{enumerate}
\end{Definition}
\begin{Definition}
	A \textit{subspace} of a vector space $ V $ is a subset $ H $ of $ V $ that has three properties:
	\begin{itemize}
		\item The zero

## Split document into chunks

Разделим на блоки, что эффективней доставать информацию.

In [19]:
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

In [20]:
len(chunks)

32

In [21]:
def get_text_embedding(input):
    embeddings_batch_response = client.embeddings.create(
          model="mistral-embed",
          inputs=input
      )
    return embeddings_batch_response.data[0].embedding

In [22]:
import time

embedddings_list = []
for chunk in chunks:
    embedddings_list.append(get_text_embedding(chunk))
    time.sleep(2)

text_embeddings = np.array(embedddings_list)

In [23]:
text_embeddings.shape

(32, 1024)

In [24]:
text_embeddings

array([[-0.02005005,  0.02178955,  0.0362854 , ..., -0.00847626,
        -0.01838684, -0.01568604],
       [-0.02038574,  0.00746155,  0.04003906, ..., -0.02381897,
        -0.00625229, -0.0329895 ],
       [-0.01922607,  0.02337646,  0.03875732, ..., -0.0027771 ,
        -0.03320312, -0.02058411],
       ...,
       [-0.02261353,  0.02246094,  0.05780029, ..., -0.01501465,
        -0.01483917, -0.02059937],
       [-0.03170776,  0.02996826,  0.0609436 , ..., -0.01190186,
        -0.03738403, -0.00258255],
       [-0.03613281,  0.02874756,  0.03875732, ..., -0.0252533 ,
        -0.01154327,  0.01005554]])

### Load into a vector database


In [25]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [60]:
def run_mistral(user_message, model="mistral-large-latest"):
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

Напишем промпты и посмотрим, что выведет моделька:

In [61]:
questions = ["Give the definition of the rank of the matrix", "Tell me about the eigenvectors", "Give the statement The Basis Theorem and explain it"]

for question in questions:
    print("#######################################")
    question_embeddings = np.array([get_text_embedding(question)])
    question_embeddings.shape
    D, I = index.search(question_embeddings, k=2)
    #print(I)
    retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
    #print(retrieved_chunk)
    prompt = f"""
    Context information is below.
    ---------------------
    {retrieved_chunk}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {question}
    Answer:
    """
    time.sleep(2)
    print(run_mistral(prompt))
    time.sleep(2)

#######################################
The rank of a matrix \( \matr{A} \), denoted by \( \rank \matr{A} \), is the dimension of the column space of \( \matr{A} \).
#######################################
An eigenvector of an \( n \times n \) matrix \(\mathbf{A}\) is a nonzero vector \(\vec{x}\) such that \(\mathbf{A} \vec{x} = \lambda \vec{x}\) for some scalar \(\lambda\). Here, \(\lambda\) is called an eigenvalue corresponding to the eigenvector \(\vec{x}\).

The set of all solutions to the equation \((\mathbf{A} - \lambda \mathbf{I}) \vec{x} = \vec{0}\) forms a subspace of \(\mathbb{R}^n\) known as the eigenspace of \(\mathbf{A}\) corresponding to the eigenvalue \(\lambda\).

Eigenvectors corresponding to distinct eigenvalues of a matrix are linearly independent. This means that if \(\vec{v}_1, \vec{v}_2, \ldots, \vec{v}_r\) are eigenvectors corresponding to distinct eigenvalues \(\lambda_1, \lambda_2, \ldots, \lambda_r\) of an \(n \times n\) matrix \(\mathbf{A}\), then the set \(\